In [69]:
import email, smtplib, ssl
import scrapy
import os
import glob
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from io import StringIO
from pdfminer.pdfpage import PDFPage
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#DEF do leitor de PDF
def get_pdf_file_content(path_to_pdf):
    
    '''
    path_to_pdf: is the parameter that will give access to the PDF File 
    we want to extract the content.
    '''
    
    '''
    PDFResourceManager is used to store shared resources such as fonts or images that 
    we might encounter in the files. 
    '''
    
    resource_manager = PDFResourceManager(caching=True)
    
    '''
    create a string object that will contain the final text the representation of the pdf. 
    '''
    out_text = StringIO()
    
    '''
    UTF-8 is one of the most commonly used encodings, and Python often defaults to using it.
    In our case, we are going to specify in order to avoid some encoding errors.
    '''
    #codec = 'utf-8'
    
    """
    LAParams is the object containing the Layout parameters with a certain default value. 
    """
    laParams = LAParams()
    
    '''
    Create a TextConverter Object, taking :
    - ressource_manager,
    - out_text 
    - layout parameters.
    '''
    text_converter = TextConverter(resource_manager, out_text, laparams=laParams)
    fp = open(path_to_pdf, 'rb')
    
    '''
    Create a PDF interpreter object taking: 
    - ressource_manager 
    - text_converter
    '''
    interpreter = PDFPageInterpreter(resource_manager, text_converter)

    '''
    We are going to process the content of each page of the original PDF File
    '''
    page = ""
    for page in PDFPage.get_pages(fp, pagenos=set(), maxpages=0, password="", caching=True, check_extractable=True):
        interpreter.process_page(page)

    '''
    Retrieve the entire contents of the “file” at any time 
    before the StringIO object’s close() method is called.
    '''
    text = out_text.getvalue()

    '''
    Closing all the ressources we previously opened
    '''
    fp.close()
    text_converter.close()
    out_text.close()
    
    '''
    Return the final variable containing all the text of the PDF
    '''
    return text

In [70]:
try:
    os.remove("texto2.txt")
except:
    pass
try:
    os.remove("extrato2.txt")
except:
    pass
try:
    os.remove("doe.pdf")
except:
    pass
try:
    os.remove("texto.txt")
except:
    pass
try:
    os.remove("extrato.txt")
except:
    pass
try:
    os.remove("doe2.pdf")
except:
    pass

#Executando os dois crawlers
os.system(r'scrapy crawl doesc')

0

In [71]:
#renomeando os dois arquivos PDF
arquivos = []
arquivos = glob.glob("*.pdf")
os.rename(arquivos[0],"doe.pdf")

In [72]:
#definindo variaveis para extração dos textos com o DEF acima
path_to_pdf = "doe.pdf"
#extraindo texto do PDF 1
texto = get_pdf_file_content(path_to_pdf)

In [73]:
#Salvando texto em arquivo TXT
with open("texto.txt", "w", encoding="utf-8") as g:
    g.write((str(texto).upper()))

#Abrindo arquivo e inserindo cada linha do arquivo numa posição em uma lista
g = open('texto.txt', 'r', encoding="utf-8")
lista = g.readlines()

In [74]:
indices = []
for i in range(len(lista)):
   if lista[i] == (f'DIÁRIO OFICIAL - SC - Nº {lista[2].strip(" NÚMERO")}'):
      indices.append(i)

In [75]:
for cadaindice in reversed(indices):
    lista.pop(cadaindice-2)

In [76]:
indices = []
for i in range(len(lista)):
   if lista[i] == (f'DIÁRIO OFICIAL - SC - Nº {lista[2].strip(" NÚMERO")}'):
      indices.append(i)

In [77]:
for cadaindice in reversed(indices):
    lista.pop(cadaindice+2)

In [78]:
indices = []
for i in range(len(lista)):
   if lista[i] == (f'DIÁRIO OFICIAL - SC - Nº {lista[2].strip(" NÚMERO")}'):
      indices.append(i)

In [79]:
for cadaindice in reversed(indices):
    lista.pop(cadaindice)

In [80]:
texto = ' '.join([str(item) for item in lista])

In [81]:
#Salvando texto em arquivo TXT
with open("texto.txt", "w", encoding="utf-8") as g:
    g.write((str(texto).upper()))

#Abrindo arquivo e inserindo cada linha do arquivo numa posição em uma lista
g = open('texto.txt', 'r', encoding="utf-8")
lista = g.readlines()

In [82]:
#Procurando palavras chaves no PDF1 e criando um arquivo somente com os resultados
search_keywords=["SAP", "PENITENCIÁRIA INDUSTRIAL DE BLUMENAU", "PRESÍDIO REGIONAL DE BLUMENAU"]
g = open('texto.txt', 'r', encoding="utf-8")
for sentence in g:
    lst = []
    for word in search_keywords:
        if word in sentence:
            lst.append(word)
            ex = open("extrato.txt", "a", encoding="utf-8")
            if "\n" in lista[(lista.index(sentence)-1)]:
                ex.write('<h3 style="color:red;">Palavra-chave Encontrada: {0}</h3>'.format(', '.join(lst)))
                ex.write( "".join(lista[(lista.index(sentence)):(lista.index(sentence)+15)]))
                ex.write("<br>")
                ex.write("<br>")
                ex.write("<br>")
                ex.write("<br>")
            else:
                ex.write('<h3 style="color:red;">Palavra-chave Encontrada: {0}</h3>'.format(', '.join(lst)))
                ex.write("".join(lista[(lista.index(sentence)-15):(lista.index(sentence)+1)]))
                ex.write("<br>")
                ex.write("<br>")
                ex.write("<br>")
                ex.write("<br>")

#Enviando Email com os resultados da analise do PDF1
ax = open("extrato.txt", "r", encoding="utf-8")

subject = 'Verificação DOE: {}'.format(" ".join(lista[0:3]))
body = ax.read()
sender_email = "bolsista@scpar.sc.gov.br"
receiver_email = "bolsista@scpar.sc.gov.br"
password = "@jld456VV"

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message["Bcc"] = receiver_email  # Recommended for mass emails

# Add body to email
message.attach(MIMEText(body, "html"))

#filename = ""  # In same directory as script

# Open PDF file in binary mode
#with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
#part = MIMEBase("application", "octet-stream")
#part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
#encoders.encode_base64(part)

# Add header as key/value pair to attachment part
#part.add_header(
#    "Content-Disposition",
#    f"attachment; filename= {filename}",
#)

# Add attachment to message and convert message to string
#message.attach(part)
text = message.as_string()

# Log in to server using secure context and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)

In [84]:
ex.close()
ax.close()
g.close()